**Question 1: Speech Enhancement**

# Installations

In [ ]:
!pip install datasets
!pip install pesq
!pip install tqdm
!pip install pydub
!pip install pydub librosa numpy soundfile
!pip install mir_eval
!pip install torchmetrics
!pip install speechbrain
!pip install pydub
!pip install pydub librosa numpy soundfile
!pip install mir_eval
!pip install torchmetrics
!pip install speechbrain

# Imports

In [3]:
from transformers import Wav2Vec2Processor, HubertForCTC, Wav2Vec2Model, HubertModel, WavLMModel
from datasets import load_dataset
from google.colab import drive
import torchaudio
import os
import torch
import torchaudio
from torch.utils.data import DataLoader, Dataset
from peft import LoraConfig, get_peft_model
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import roc_curve
from tqdm import tqdm
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from mir_eval.separation import bss_eval_sources
from pesq import pesq
from pydub import AudioSegment
import librosa
import numpy as np
import soundfile as sf
import random

# I. Download Dataset

In [4]:

drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
vox1_zip_path = '/content/drive/MyDrive/vox1/vox1_test_wav.zip'
vox2_zip_path = '/content/drive/MyDrive/vox2/vox2_test_aac.zip'
vox2_text_zip_path = '/content/drive/MyDrive/vox2/vox2_test_txt.zip'

In [6]:
vox1_data = '/content/vox1/wav'
vox2_data = '/content/vox2/acc'
vox2_text = '/content/vox2/txt'

In [ ]:
!unzip /content/drive/MyDrive/vox1/vox1_test_wav.zip -d /content/vox1

In [ ]:
!unzip /content/drive/MyDrive/vox2/vox2_test_aac.zip -d /content/vox2

In [ ]:
!unzip /content/drive/MyDrive/vox2/vox2_test_txt.zip -d /content/vox2

In [ ]:
!wget https://mm.kaist.ac.kr/datasets/voxceleb/meta/veri_test2.txt

# II. Speaker Verification ( pretrained vs peft lora model )

### Hubert Large Pretrained

In [11]:
processor = Wav2Vec2Processor.from_pretrained("facebook/hubert-large-ls960-ft")
model = HubertForCTC.from_pretrained("facebook/hubert-large-ls960-ft")

ds = load_dataset("patrickvonplaten/librispeech_asr_dummy", "clean", split="validation")

input_values = processor(ds[0]["audio"]["array"], return_tensors="pt").input_values  # Batch size 1
logits = model(input_values).logits
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.decode(predicted_ids[0])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

librispeech_asr_dummy.py:   0%|          | 0.00/5.16k [00:00<?, ?B/s]

The repository for patrickvonplaten/librispeech_asr_dummy contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/patrickvonplaten/librispeech_asr_dummy.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating validation split: 0 examples [00:00, ? examples/s]

It is strongly recommended to pass the `sampling_rate` argument to `Wav2Vec2FeatureExtractor()`. Failing to do so can result in silent errors that might be hard to debug.


### Wavlm Base Plus Pretrained

In [12]:
# Configurations
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_PATH = "/content/saved_models/wavlm_epoch_3.pt"
processor = Wav2Vec2Processor.from_pretrained("facebook/hubert-large-ls960-ft")

wavlm_model = WavLMModel.from_pretrained("microsoft/wavlm-base-plus").to(DEVICE)
wavlm_model.eval()


config.json:   0%|          | 0.00/2.23k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/378M [00:00<?, ?B/s]

WavLMModel(
  (feature_extractor): WavLMFeatureEncoder(
    (conv_layers): ModuleList(
      (0): WavLMGroupNormConvLayer(
        (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
        (activation): GELUActivation()
        (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
      )
      (1-4): 4 x WavLMNoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
      (5-6): 2 x WavLMNoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
    )
  )
  (feature_projection): WavLMFeatureProjection(
    (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (projection): Linear(in_features=512, out_features=768, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): WavLMEncoder(
    (pos_conv_embed): WavLMPositionalConvEmbedding(
      (conv): Parametrized

In [14]:


def extract_embedding(file_path, model):
    """Load an audio file and extract its speaker embedding."""
    waveform, sample_rate = torchaudio.load(file_path)

    # Resample if needed
    if sample_rate != 16000:
        waveform = torchaudio.transforms.Resample(sample_rate, 16000)(waveform)

    # Convert to mono
    if waveform.shape[0] > 1:
        waveform = waveform.mean(dim=0, keepdim=True)

    # Process input
    inputs = processor(waveform.squeeze(0), sampling_rate=16000, return_tensors="pt", padding=True)
    inputs = inputs.input_values.to(DEVICE)

    # Get embeddings
    with torch.no_grad():
        outputs = model(inputs).last_hidden_state
        embedding = outputs.mean(dim=1)  # Mean pooling

    return embedding.cpu()  # Return embedding as a CPU tensor


from sklearn.metrics.pairwise import cosine_similarity

file1 = "/content/vox1/wav/id10270/x6uYqmx31kE/00001.wav"
file2 = "/content/vox1/wav/id10270/8jEAjG6SegY/00008.wav"

embedding1 = extract_embedding(file1, wavlm_model)
embedding2 = extract_embedding(file2, wavlm_model)

similarity = cosine_similarity(embedding1.numpy(), embedding2.numpy())[0][0]
print(f"Speaker Similarity: {similarity:.4f}")


Speaker Similarity: 0.9215


### LORA PEFT Pretrained Wavlm Base Plus Hugging Face Trainer

#### Configuration

In [ ]:
# ---- CONFIGURATION ----
DATASET_PATH = "/content/content/vox2/aac"
TRIAL_FILE = "/content/veri_test2.txt"
BATCH_SIZE = 8
EPOCHS = 5
LR = 1e-6
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


#### Pre Trained Model

In [ ]:


# ---- LOAD PRE-TRAINED MODEL ----
processor = Wav2Vec2Processor.from_pretrained("facebook/hubert-large-ls960-ft")
model = WavLMModel.from_pretrained("microsoft/wavlm-base-plus").to(DEVICE)


#### LORA Config

In [ ]:

# ---- APPLY LoRA ----
lora_config = LoraConfig(
    r=8, lora_alpha=32, target_modules=[
        "encoder.layers.11.attention.q_proj",
        "encoder.layers.11.attention.k_proj",
        "encoder.layers.11.attention.v_proj",
        "encoder.layers.11.attention.o_proj",
        "encoder.layers.11.feed_forward.fc_in",
        "encoder.layers.11.feed_forward.fc_out"
    ], lora_dropout=0.1
)
model = get_peft_model(model, lora_config)
model.to(DEVICE)


#### VoxCeleb Dataset

In [ ]:

# ---- DEFINE DATASET ----
# ---- CREATE GLOBAL SPEAKER MAPPING ----
all_speakers = sorted(os.listdir(DATASET_PATH))
speaker_to_id = {spk: i for i, spk in enumerate(all_speakers)}  # Assign unique IDs globally
id_to_speaker = {v: k for k, v in speaker_to_id.items()}  # Inverse mapping

train_speakers = all_speakers[:100]
test_speakers = all_speakers[100:118]

# ---- MODIFY DATASET CLASS TO USE GLOBAL MAPPING ----
class VoxCelebDataset(Dataset):
    def __init__(self, dataset_path, speakers, speaker_to_id, processor, max_files_per_speaker=20):
        self.data = []
        self.processor = processor
        self.speaker_to_id = speaker_to_id  # Use global mapping

        for speaker in speakers:
            speaker_path = os.path.join(dataset_path, speaker)
            speaker_files = []

            for scene in os.listdir(speaker_path):
                scene_path = os.path.join(speaker_path, scene)
                if os.path.isdir(scene_path):
                    m4a_files = [os.path.join(scene_path, f) for f in sorted(os.listdir(scene_path)) if f.endswith(".m4a")]
                    speaker_files.extend(m4a_files)

            speaker_files = speaker_files[:max_files_per_speaker]

            for audio_file in speaker_files:
                self.data.append((audio_file, speaker))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        audio_path, speaker = self.data[idx]
        waveform, sample_rate = torchaudio.load(audio_path)

        if sample_rate != 16000:
            waveform = torchaudio.transforms.Resample(sample_rate, 16000)(waveform)

        if waveform.shape[0] > 1:
            waveform = waveform.mean(dim=0)

        inputs = self.processor(waveform, sampling_rate=16000, return_tensors="pt", padding=True)

        return {
            "input_values": inputs.input_values.squeeze(0),
            "labels": self.speaker_to_id[speaker],  # Convert labels to tensor
            "speaker_name": speaker  # Store speaker name for later use
        }


#### Data Collator

In [ ]:


# ---- LOAD DATA WITH GLOBAL SPEAKER ID ----
train_dataset = VoxCelebDataset(DATASET_PATH, train_speakers, speaker_to_id, processor)
test_dataset = VoxCelebDataset(DATASET_PATH, test_speakers, speaker_to_id, processor)


class DataCollator:
    def __call__(self, batch):
        inputs = [b["input_values"] for b in batch if "input_values" in b]
        labels = [b["labels"] for b in batch if "labels" in b]

        if not inputs or not labels:
            raise ValueError("Missing input_values or labels in batch.")

        max_len = max(inp.shape[-1] for inp in inputs)
        inputs = [F.pad(inp, (0, max_len - inp.shape[-1])) for inp in inputs]

        inputs = torch.stack(inputs, dim=0)
        labels = torch.tensor(labels, dtype=torch.long)  # Ensure it's a LongTensor

        return {"input_values": inputs, "labels": labels}


#### LOSS Function

In [ ]:

# ---- DEFINE CUSTOM LOSS FUNCTION ----
class ArcFaceLoss(torch.nn.Module):
    def __init__(self, num_classes, embedding_dim, scale=30, margin=0.5):
        super().__init__()
        self.W = torch.nn.Parameter(torch.randn(embedding_dim, num_classes))
        self.scale = scale
        self.margin = margin

    def forward(self, embeddings, labels):
        W_norm = F.normalize(self.W, dim=0)
        embeddings_norm = F.normalize(embeddings, dim=1)
        cos_theta = torch.matmul(embeddings_norm, W_norm)

        target_logits = cos_theta[torch.arange(len(labels)), labels]
        modified_logits = torch.cos(torch.acos(target_logits) + self.margin)
        cos_theta[torch.arange(len(labels)), labels] = modified_logits

        return F.cross_entropy(self.scale * cos_theta, labels)


#### Customer Trainer

In [ ]:


# ---- TRAINER WRAPPER ----
class CustomTrainer(Trainer):
    def __init__(self, arcface_loss, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.arcface_loss = arcface_loss

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None, labels = None):
        labels = inputs.pop("labels")
        input_values = inputs.pop("input_values")
        outputs = model(input_values.squeeze(1)).last_hidden_state.mean(dim=1)
        loss = self.arcface_loss(outputs, labels.to(DEVICE))
        return (loss, outputs) if return_outputs else loss

# ---- SET TRAINING ARGUMENTS ----
training_args = TrainingArguments(
    output_dir="/content/saved_models",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=LR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    gradient_accumulation_steps=1,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    # save_total_limit=2,
    fp16=True if DEVICE == "cuda" else False,
    remove_unused_columns=False,
)



#### Initialize Trainer

In [ ]:
# ---- INITIALIZE TRAINER ----
arcface_loss = ArcFaceLoss(num_classes=len(train_speakers), embedding_dim=768).to(DEVICE)

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    processing_class=processor,  # Replace `tokenizer` with `processing_class`
    data_collator=DataCollator(),
    # label_names=["labels"],
    arcface_loss=arcface_loss
)

# ---- TRAIN MODEL ----
trainer.train()

# Evaluate the fine-tuned model
results = trainer.evaluate()

# Print evaluation results
print(results)
# ---- SAVE MODEL ----
trainer.save_model("/content/saved_models/wavlm_trained")

### Evaluation

In [16]:
def read_trial_pairs(file_path):
    """Read trial pairs from a text file."""
    trials = []
    with open(file_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) == 3:
                label, file1, file2 = parts
                print(label)
                print(file1)
                print(file2)
                trials.append((int(label), f"/content/vox1/wav/{file1}", f"/content/vox1/wav/{file2}"))
    return trials

In [22]:
# Load the pre-trained HuBERT Large model
processor = Wav2Vec2Processor.from_pretrained("facebook/hubert-large-ls960-ft")
model = WavLMModel.from_pretrained("microsoft/wavlm-base-plus").to(DEVICE)

def extract_embedding(audio_path):
    # Load audio file
    waveform, sample_rate = torchaudio.load(audio_path)

    if sample_rate != 16000:
        waveform = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)(waveform)

    if waveform.shape[0] > 1:
        waveform = waveform.mean(dim=0, keepdim=True)
    if torch.all(waveform == 0):
        return torch.zeros((1, 1024))

    # Process input
    inputs = processor(waveform.squeeze(0), sampling_rate=16000, return_tensors="pt", padding=True)

    # Get hidden states from the model
    with torch.no_grad():
        outputs = model(**inputs)

    # Compute mean embedding
    embedding = outputs.last_hidden_state.mean(dim=1)

    # Check for NaN values
    if torch.isnan(embedding).any():
        return torch.zeros((1, 1024))

    return embedding


# Load trial pairs (Modify this to read from veri_test2.txt)
voxceleb1_cleaned = read_trial_pairs("/content/veri_test2.txt")

results = []
for label, audio_file1, audio_file2 in voxceleb1_cleaned[:1]:  # Use 100 samples for comparison
    embedding1 = extract_embedding(audio_file1)
    embedding2 = extract_embedding(audio_file2)

    # Compute cosine similarity
    similarity_score = cosine_similarity(embedding1.numpy(), embedding2.numpy())[0][0]

    results.append((similarity_score, label))

# Convert to numpy arrays
scores, labels = zip(*results)
scores = np.array(scores)
labels = np.array(labels)

# Compute EER and TAR@1%FAR
fpr, tpr, thresholds = roc_curve(labels, scores)
eer = fpr[np.argmin(np.abs(fpr - (1 - tpr)))]
tar_1_far = tpr[np.argmin(np.abs(fpr - 0.01))]

# Calculate Accuracy
threshold = 0.5  # Adjust based on dataset
correct = sum(1 for score, label in results if (score > threshold) == label)
accuracy = correct / len(results)

# Print Results
print(f"Pretrained Speaker Verification Results:")
print(f"EER: {eer:.2%}")
print(f"TAR@1%FAR: {tar_1_far:.2%}")
print(f"Speaker Identification Accuracy: {accuracy:.2%}")


Pretrained Speaker Verification Results:
EER: 8.2
TAR@1%FAR: 0.63
Speaker Identification Accuracy: 72.00%


# Common for Q1 III and Q1 IV

## Loss Metric for Speech Enhancement Pipeline

In [ ]:
def evaluate_audio(true_sources, predicted_sources, original_sample_rate=16000, target_sample_rate=8000):
    """
    Evaluates speech enhancement using SDR, SIR, SAR, and PESQ metrics.
    Resamples audio from the original sample rate to the target sample rate if necessary.
    """
    # Ensure true_sources and predicted_sources are PyTorch tensors
    true_sources = true_sources.detach().cpu()
    predicted_sources = predicted_sources.detach().cpu()

    # Resample both true_sources and predicted_sources from 16kHz to 8kHz (or any other target rate)
    resampler = torchaudio.transforms.Resample(orig_freq=original_sample_rate, new_freq=target_sample_rate)

    # Resample
    true_sources_resampled = resampler(true_sources)
    predicted_sources_resampled = resampler(predicted_sources)

    # Check if padding is needed (pad the smaller tensor)
    true_len = true_sources_resampled.shape[-1]
    pred_len = predicted_sources_resampled.shape[-1]

    if true_len > pred_len:
        # Pad the predicted_sources_resampled tensor to match the true_sources_resampled length
        padding_amount = true_len - pred_len
        predicted_sources_resampled = F.pad(predicted_sources_resampled, (0, padding_amount))
    elif pred_len > true_len:
        # Pad the true_sources_resampled tensor to match the predicted_sources_resampled length
        padding_amount = pred_len - true_len
        true_sources_resampled = F.pad(true_sources_resampled, (0, padding_amount))

    # Ensure that both tensors are reshaped to (num_sources, num_frames)
    true_sources_resampled = true_sources_resampled.reshape(true_sources_resampled.shape[0], -1)
    predicted_sources_resampled = predicted_sources_resampled.reshape(predicted_sources_resampled.shape[0], -1)

    # Compute BSS eval metrics
    sdr, sir, sar, _ = bss_eval_sources(true_sources_resampled.numpy(), predicted_sources_resampled.numpy())

    # Select PESQ mode based on sample rate
    pesq_mode = "wb" if target_sample_rate == 16000 else "nb"

    # Compute PESQ score (for one source, we select the first one in the batch)
    pesq_score = pesq(target_sample_rate, true_sources_resampled[0].numpy(), predicted_sources_resampled[0].numpy(), pesq_mode)

    return {"SDR": sdr, "SIR": sir, "SAR": sar, "PESQ": pesq_score}

def combined_loss(separated_audio, true_sources, predicted_embeddings, true_embeddings):
    """
    Computes the combined enhancement + speaker identification loss.
    """
    # Speech Enhancement Loss (SDR)
    enhancement_metrics = evaluate_audio(true_sources, separated_audio)
    sdr_loss = -torch.tensor(enhancement_metrics["SDR"]).mean()  # Maximizing SDR (negating for loss)

    # Speaker Identification Loss (Cosine Similarity)
    # Flatten the embeddings if they have extra dimensions
    predicted_embeddings_flat = predicted_embeddings.view(predicted_embeddings.size(0), -1)
    true_embeddings_flat = true_embeddings.view(true_embeddings.size(0), -1)

    # Ensure the target is a 1D tensor with the correct shape
    target = torch.ones(predicted_embeddings_flat.size(0)).to(predicted_embeddings_flat.device)

    # Compute the cosine embedding loss
    identification_loss = F.cosine_embedding_loss(predicted_embeddings_flat, true_embeddings_flat, target)

    # Weighted sum of losses
    lambda1, lambda2 = 0.7, 0.3
    total_loss = lambda1 * identification_loss + lambda2 * sdr_loss

    return total_loss, enhancement_metrics




# III . Creating Multi Speaker Scenario And Evaluating Pretrained Model

## Preparing Dataset for MultiSpeaker Speech Enhancement

### Classifying first 50 for training, next 50 for testing

In [ ]:
import os

vox_dir = "/content/vox2/aac"
speakers = sorted(os.listdir(vox_dir))  # Sort speaker IDs in ascending order

# Select first 50 for training and next 50 for testing
train_speakers = speakers[:50]
test_speakers = speakers[50:100]

print("Training speakers:", train_speakers)
print("Testing speakers:", test_speakers)

Training speakers: ['id00017', 'id00061', 'id00081', 'id00154', 'id00419', 'id00562', 'id00812', 'id00817', 'id00866', 'id00926', 'id01000', 'id01041', 'id01066', 'id01106', 'id01224', 'id01228', 'id01298', 'id01333', 'id01437', 'id01460', 'id01509', 'id01541', 'id01567', 'id01593', 'id01618', 'id01822', 'id01892', 'id01989', 'id02019', 'id02057', 'id02086', 'id02181', 'id02286', 'id02317', 'id02445', 'id02465', 'id02542', 'id02548', 'id02576', 'id02577', 'id02685', 'id02725', 'id02745', 'id03030', 'id03041', 'id03127', 'id03178', 'id03347', 'id03382', 'id03524']
Testing speakers: ['id03677', 'id03789', 'id03839', 'id03862', 'id03969', 'id03978', 'id03980', 'id03981', 'id04006', 'id04030', 'id04094', 'id04119', 'id04232', 'id04253', 'id04276', 'id04295', 'id04366', 'id04478', 'id04536', 'id04570', 'id04627', 'id04656', 'id04657', 'id04862', 'id04950', 'id05015', 'id05055', 'id05124', 'id05176', 'id05202', 'id05459', 'id05594', 'id05654', 'id05714', 'id05816', 'id05850', 'id05999', 'id0

### Converting m4a -> wav

In [ ]:

def convert_m4a_to_wav(m4a_path, wav_path):
    audio = AudioSegment.from_file(m4a_path, format="m4a")
    audio.export(wav_path, format="wav")

### Mixing two audio s

In [ ]:


def mix_audio(file1, file2, output_path, snr=0):
    y1, sr1 = librosa.load(file1, sr=None)
    y2, sr2 = librosa.load(file2, sr=None)

    # # Ensure same sampling rate
    # assert sr1 == sr2, "Sampling rates must match."

    # # Truncate to same length
    # min_len = min(len(y1), len(y2))
    # y1, y2 = y1[:min_len], y2[:min_len]

    # # Normalize volume
    # y1 /= np.max(np.abs(y1))
    # y2 /= np.max(np.abs(y2))

    # # Mix with given SNR
    # mix = y1 + (y2 / (10**(snr / 20)))
    if sr1 != sr2:
        min_sr = min(sr1, sr2)
        y1 = librosa.resample(y1, orig_sr=sr1, target_sr=min_sr)
        y2 = librosa.resample(y2, orig_sr=sr2, target_sr=min_sr)
        sr = min_sr
    else:
        sr = sr1

    min_len = min(len(y1), len(y2))
    y1, y2 = y1[:min_len], y2[:min_len]

    mixed_audio = 0.5 * y1 + 0.5 * y2

    # Save mixed audio
    sf.write(output_path, mixed_audio, sr)

    return output_path

# Example usage
mix_audio("/content/vox2/aac/id00017/01dfn2spqyE/00001.m4a", "/content/vox2/aac/id00081/2xYrsnvtUWc/00001.m4a", "mixed_output.wav")


### Mix 1000 files for training and remaining for testing

In [ ]:
import random

output_dir = "multi_speaker_dataset"
os.makedirs(output_dir, exist_ok=True)

def get_all_audio_files(speaker_dir):
    """Recursively get all .m4a files for a given speaker."""
    audio_files = []
    for root, _, files in os.walk(speaker_dir):
        for file in files:
            if file.endswith(".m4a"):
                audio_files.append(os.path.join(root, file))
    return audio_files

def create_multi_speaker_dataset(speaker_list, split, limit):
    data = []
    os.makedirs(os.path.join(output_dir, split), exist_ok=True)
    for _ in range(limit):  # Generate 1000 mixed samples
        spk1, spk2 = random.sample(speaker_list, 2)  # Pick 2 random speakers

        spk1_dir = os.path.join(vox_dir, spk1)
        spk2_dir = os.path.join(vox_dir, spk2)

        spk1_files = get_all_audio_files(spk1_dir)
        spk2_files = get_all_audio_files(spk2_dir)

        if not spk1_files or not spk2_files:
            continue

        file1 = random.choice(spk1_files)
        file2 = random.choice(spk2_files)

        # Mix and save
        mixed_output = f"{output_dir}/{split}/{spk1}_{spk2}_mix.wav"
        mix_audio(file1, file2, mixed_output)
        data.append({"speaker1": spk1, "speaker2": spk2, "mixed_audio": mixed_output, "speaker1_path": file1, "speaker2_path": file2})

    return data

train_data = create_multi_speaker_dataset(train_speakers, "train", 2000)
test_data = create_multi_speaker_dataset(test_speakers, "test", 200)


In [ ]:
import pandas as pd

# Convert to DataFrame
train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)

# Save to CSV
train_df.to_csv("train_data_multi_mix.csv", index=False)
test_df.to_csv("test_data_multi_mix.csv", index=False)


In [ ]:
!zip -r multi_speaker_dataset.zip multi_speaker_dataset


In [ ]:
from google.colab import files
files.download("multi_speaker_dataset.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Loading Models(pretrained) - Speaker Identification & Speech Enhacement Model

In [26]:
import librosa
import torch

def load_mixed_audio(file_path):
    waveform, sr = librosa.load(file_path, sr=16000)
    return torch.tensor(waveform), sr


In [27]:
import torch
from transformers import WavLMModel, Wav2Vec2Processor
from peft import PeftModel, LoraConfig, get_peft_model

# Configurations
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Load processor
processor = Wav2Vec2Processor.from_pretrained("facebook/hubert-large-ls960-ft")

# Load the base model
identification_model = WavLMModel.from_pretrained("microsoft/wavlm-base-plus").to(DEVICE)
identification_model.eval()
print("WavLM model loaded successfully!")


WavLM model loaded successfully!


In [28]:


import torchaudio

def extract_embedding(file_path, model):
    """Load an audio file and extract its speaker embedding."""
    waveform, sample_rate = torchaudio.load(file_path)

    # Resample if needed
    if sample_rate != 16000:
        waveform = torchaudio.transforms.Resample(sample_rate, 16000)(waveform)

    # Convert to mono
    if waveform.shape[0] > 1:
        waveform = waveform.mean(dim=0, keepdim=True)

    # Process input
    inputs = processor(waveform.squeeze(0), sampling_rate=16000, return_tensors="pt", padding=True)
    inputs = inputs.input_values.to(DEVICE)

    # Get embeddings
    with torch.no_grad():
        outputs = model(inputs).last_hidden_state
        embedding = outputs.mean(dim=1)  # Mean pooling

    return embedding.cpu()  # Return embedding as a CPU tensor


from sklearn.metrics.pairwise import cosine_similarity

file1 = "/content/vox1/wav/id10270/x6uYqmx31kE/00001.wav"
file2 = "/content/vox1/wav/id10270/8jEAjG6SegY/00008.wav"

embedding1 = extract_embedding(file1, identification_model)
embedding2 = extract_embedding(file2, identification_model)

similarity = cosine_similarity(embedding1.numpy(), embedding2.numpy())[0][0]
print(f"Speaker Similarity: {similarity:.4f}")


Speaker Similarity: 0.9215


In [29]:
from speechbrain.inference.separation import SepformerSeparation as separator
import torchaudio

sepformer_model = separator.from_hparams(source="speechbrain/sepformer-whamr", savedir='pretrained_models/sepformer-whamr')

def separate_audio_sources(path, model, target_sample_rate=16000):
    """
    Separates mixed audio into two sources and ensures they are at the target sample rate (16000 Hz).
    Returns the separated audio tensors at 16000 Hz.
    """
    # Load the mixed audio file
    waveform, sample_rate = torchaudio.load(path)

    # If the sample rate is not the target, resample the audio
    if sample_rate != target_sample_rate:
        resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=target_sample_rate)
        waveform = resampler(waveform)

    # Perform the separation
    est_sources = model.separate_batch(waveform)  # Separate sources
    source1 = est_sources[:, :, 0].detach().cpu()
    source2 = est_sources[:, :, 1].detach().cpu()

    # Optionally, if you need to resample the separated audio to 16000 Hz (ensure it matches the desired sample rate)
    resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=target_sample_rate)
    source1_resampled = resampler(source1)  # Resample the first source if needed
    source2_resampled = resampler(source2)  # Resample the second source if needed

    return source1_resampled, source2_resampled  # Returning the separated audio tensors at 16000 Hz


# speech_separation('/content/multi_speaker_dataset/multi_speaker_dataset/test/id03677_id05202_mix.wav')
# for custom file, change path


hyperparams.yaml:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

masknet.ckpt:   0%|          | 0.00/113M [00:00<?, ?B/s]

encoder.ckpt:   0%|          | 0.00/17.3k [00:00<?, ?B/s]

decoder.ckpt:   0%|          | 0.00/17.3k [00:00<?, ?B/s]

In [30]:
def extract_embeddings(audio, model, processor):
  inputs = processor(audio.squeeze(0), return_tensors="pt", sampling_rate=16000)
  with torch.no_grad():
      model_output = model(**inputs)
      embeddings = model_output.last_hidden_state.mean(dim=1)

  return embeddings.cpu()  # Shape: (embedding_dim,)

In [31]:
def extract_embeddings_from_separated(audio_tensors, model, processor):
    """
    Extracts speaker embeddings from the separated audio tensors.
    Instead of returning a list of embeddings, it returns a stacked tensor.
    """
    embeddings = [extract_embeddings(audio, model, processor) for audio in audio_tensors]

    # Stack the list of embeddings into a single tensor
    stacked_embeddings = torch.stack(embeddings, dim=0)

    return stacked_embeddings

In [32]:
def enhance_and_identify_speakers(mixed_audio_path, speech_enhancement_model, speaker_identification_model, processor):
    separated_sources = separate_audio_sources(mixed_audio_path, speech_enhancement_model)
    identified_speakers = extract_embeddings_from_separated(separated_sources, speaker_identification_model, processor)
    return identified_speakers

## MutliSpeaker Dataset Loader

In [35]:
import torch
import torchaudio
import pandas as pd
from torch.utils.data import Dataset, DataLoader

class MultiSpeakerDataset(Dataset):
    def __init__(self, csv_path, processor, embedding_model, sample_rate=16000, device="cpu"):
        """
        Args:
            csv_path (str): Path to the CSV file (train.csv or test.csv).
            processor (Speech Processor): Pre-trained model processor (for feature extraction).
            embedding_model (Speaker Identification Model): Model to extract embeddings.
            sample_rate (int): Sampling rate for audio files.
        """
        self.data = pd.read_csv(csv_path)  # Load CSV
        self.processor = processor
        self.embedding_model = embedding_model
        self.sample_rate = sample_rate
        self.device = device

    def extract_embeddings(self, audio_path):
        """Extracts speaker embeddings from an audio file."""
        audio, sr = torchaudio.load(audio_path)
        if sr != self.sample_rate:
            audio = torchaudio.transforms.Resample(orig_freq=sr, new_freq=self.sample_rate)(audio)

        inputs = self.processor(audio.squeeze(0), return_tensors="pt", sampling_rate=self.sample_rate)

        # Ensure inputs are a dictionary and move to the correct device
        inputs = {k: v.to(self.device) for k, v in inputs.items()}

        with torch.no_grad():
            model_output = self.embedding_model(**inputs)
            embeddings = model_output.last_hidden_state.mean(dim=1)

        return embeddings.cpu()  # Shape: (embedding_dim,)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        """Returns a batch containing mixed audio, original sources, and embeddings."""
        row = self.data.iloc[idx]

        # Load audio
        speaker1_audio, sr1 = torchaudio.load(f'/{row["speaker1_path"]}')
        speaker2_audio, sr2 = torchaudio.load(f'/{row["speaker2_path"]}')
        # Ensure both audio files have the same sample rate (if needed)
        target_sr = self.processor.feature_extractor.sampling_rate
        if sr1 != target_sr:
            speaker1_audio = torchaudio.transforms.Resample(orig_freq=sr1, new_freq=target_sr)(speaker1_audio)
        if sr2 != target_sr:
            speaker2_audio = torchaudio.transforms.Resample(orig_freq=sr2, new_freq=target_sr)(speaker2_audio)
        max_len = max(speaker1_audio.shape[1], speaker2_audio.shape[1])
        speaker1_audio = F.pad(speaker1_audio, (0, max_len - speaker1_audio.shape[1]))
        speaker2_audio = F.pad(speaker2_audio, (0, max_len - speaker2_audio.shape[1]))

        # Extract embeddings for true speaker identities
        speaker1_embedding = self.extract_embeddings(f'/{row["speaker1_path"]}')
        speaker2_embedding = self.extract_embeddings(f'/{row["speaker2_path"]}')
        # Labels (Speaker Embeddings as Ground Truth)
        labels = torch.stack([speaker1_embedding, speaker2_embedding])  # Shape: (2, embedding_dim)

        return {
            "mixed_audio": row["mixed_audio"],#mixed_audio,
            "true_sources": torch.stack([speaker1_audio, speaker2_audio]),  # Original clean audio sources
            "labels": labels,  # Ground truth speaker embeddings
        }

def custom_collate_fn(batch):
    # Find the maximum sequence length in the batch for both mixed_audio and true_sources
    # max_len = max([item['mixed_audio'].shape[1] for item in batch])
    max_true_source_len = max([item['true_sources'][0].shape[1] for item in batch])
    padded_batch = {
        "mixed_audio": [],
        "true_sources": [],
        "labels": []
    }

    for item in batch:
        # Pad the mixed_audio if necessary
        padded_batch['mixed_audio'].append(item['mixed_audio'])

        # Pad the true_sources (speaker1_audio, speaker2_audio) tensors to max_len
        true_sources = item['true_sources']
        true_sources_padded = torch.stack([F.pad(source, (0, max_true_source_len - source.shape[1])) for source in true_sources])
        padded_batch['true_sources'].append(true_sources_padded)

        # Add the labels (embeddings) without padding
        labels = item['labels']
        padded_batch['labels'].append(labels)

    # Stack the padded data into a batch
    # padded_batch['mixed_audio'] = torch.stack(padded_batch['mixed_audio'])
    padded_batch['true_sources'] = torch.stack(padded_batch['true_sources'])
    padded_batch['labels'] = torch.stack(padded_batch['labels'])

    return padded_batch

# Create DataLoaders
def get_data_loaders(train_csv, test_csv, processor, embedding_model, batch_size=8, num_workers=2):
    """
    Creates train and test DataLoaders.
    """
    train_dataset = MultiSpeakerDataset(train_csv, processor, embedding_model)
    test_dataset = MultiSpeakerDataset(test_csv, processor, embedding_model)

    train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=2, collate_fn=custom_collate_fn)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=2, collate_fn=custom_collate_fn)
    return train_loader, test_loader

# Example usage:
train_loader, test_loader = get_data_loaders('/content/train_data_multi_mix.csv', '/content/test_data_multi_mix.csv', processor, identification_model)


## A. Evaluating with the pre trained Speech Enhancement Model - speechbrain/sepformer-whamr
## B.Identifying speakers using pretrained model and finetuned model

### Inference using speech brain on the mixed files

In [37]:
from speechbrain.inference.separation import SepformerSeparation as separator
import torchaudio

model = separator.from_hparams(source="speechbrain/sepformer-whamr", savedir='pretrained_models/sepformer-whamr')

def speech_separation(path):
    est_sources = model.separate_file(path)
    torchaudio.save("source1hat.wav", est_sources[:, :, 0].detach().cpu(), 16000)
    torchaudio.save("source2hat.wav", est_sources[:, :, 1].detach().cpu(), 16000)
speech_separation('/content/multi_speaker_dataset/test/id03677_id05850_mix.wav')


### Signal to Distortion Ratio (SDR)


In [38]:
import numpy as np

def calculate_sdr(references, estimates):
    """
    Compute SDR for audio source separation evaluation.

    Args:
        references: Clean reference signals (shape: [sources, channels, samples])
        estimates: Estimated separated signals (shape: [sources, channels, samples])

    Returns:
        SDR values in dB
    """
    delta = 1e-7  # avoid numerical errors

    num = np.sum(np.square(references), axis=(1, 2))
    den = np.sum(np.square(references - estimates), axis=(1, 2))

    num += delta
    den += delta

    return 10 * np.log10(num / den)


### Signal to Interference Ratio (SIR)


In [39]:
from mir_eval.separation import bss_eval_sources

def calculate_sir(references, estimates):
    """
    Compute SIR using BSS-Eval.

    Args:
        references: Clean reference signals
        estimates: Estimated separated signals

    Returns:
        SDR, SIR, SAR values
    """
    sdr, sir, sar = bss_eval_sources(references, estimates)
    return sir


### Signal to Artifacts Ratio (SAR)


In [40]:
def calculate_sar(references, estimates):
    """
    Compute SAR using BSS-Eval.

    Args:
        references: Clean reference signals
        estimates: Estimated separated signals

    Returns:
        SAR values
    """
    sdr, sir, sar = bss_eval_sources(references, estimates)
    return sar


### Perceptual Evaluation of Speech Quality (PESQ)


In [41]:
import torch
from torchmetrics.functional.audio import perceptual_evaluation_speech_quality

def calculate_pesq(reference, degraded, sample_rate=16000, mode='wb'):
    """
    Compute PESQ score.

    Args:
        reference: Clean reference signal (tensor)
        degraded: Processed/degraded signal (tensor)
        sample_rate: Sampling frequency (16000 for wideband or 8000 for narrowband)
        mode: 'wb' for wideband or 'nb' for narrowband

    Returns:
        PESQ score
    """
    # Ensure inputs are torch tensors
    if not isinstance(reference, torch.Tensor):
        reference = torch.tensor(reference)
    if not isinstance(degraded, torch.Tensor):
        degraded = torch.tensor(degraded)

    return perceptual_evaluation_speech_quality(
        degraded, reference, sample_rate, mode
    )


### All Metrics Implementation

In [42]:
!pip install pesq

In [43]:
import numpy as np
import torch
from mir_eval.separation import bss_eval_sources
from pesq import pesq

def evaluate_separation_metrics(references, estimates, sample_rate=16000):
    """
    Evaluate source separation performance using multiple metrics.

    Args:
        references: Clean reference signals
        estimates: Estimated separated signals
        sample_rate: Audio sampling rate

    Returns:
        Dictionary of metrics
    """
    # Convert to numpy arrays if they're torch tensors
    if isinstance(references, torch.Tensor):
        references_np = references.cpu().numpy()
    else:
        references_np = np.array(references)

    if isinstance(estimates, torch.Tensor):
        estimates_np = estimates.cpu().numpy()
    else:
        estimates_np = np.array(estimates)

    # Calculate BSS-Eval metrics
    sdr, sir, sar = bss_eval_sources(references_np, estimates_np)

    # Calculate PESQ for each source
    pesq_scores = []
    for i in range(len(references_np)):
        ref = torch.tensor(references_np[i])
        est = torch.tensor(estimates_np[i])

        # PESQ requires mono audio
        if len(ref.shape) > 1 and ref.shape[0] > 1:
            ref = ref[0]  # Take first channel
        if len(est.shape) > 1 and est.shape[0] > 1:
            est = est[0]  # Take first channel

        try:
            pesq_mode = "wb" if sample_rate == 16000 else "nb"
            pesq_score = pesq(sample_rate, ref, est, 'wb')
            # pesq = perceptual_evaluation_speech_quality(est, ref, sample_rate, 'wb')
            pesq_scores.append(pesq_score.item())
        except Exception as e:
            print(f"PESQ calculation failed: {e}")
            pesq_scores.append(None)

    return {
        'SDR': sdr.tolist(),
        'SIR': sir.tolist(),
        'SAR': sar.tolist(),
        'PESQ': pesq_scores
    }

def evaluate_audio(true_sources, predicted_sources, original_sample_rate=16000, target_sample_rate=8000):
    """
    Evaluates speech enhancement using SDR, SIR, SAR, and PESQ metrics.
    Resamples audio from the original sample rate to the target sample rate if necessary.
    """
    # Ensure true_sources and predicted_sources are PyTorch tensors
    true_sources = true_sources.detach().cpu()
    predicted_sources = predicted_sources.detach().cpu()

    # Resample both true_sources and predicted_sources from 16kHz to 8kHz (or any other target rate)
    resampler = torchaudio.transforms.Resample(orig_freq=original_sample_rate, new_freq=target_sample_rate)

    # Resample
    true_sources_resampled = resampler(true_sources)
    predicted_sources_resampled = resampler(predicted_sources)

    # Check if padding is needed (pad the smaller tensor)
    true_len = true_sources_resampled.shape[-1]
    pred_len = predicted_sources_resampled.shape[-1]

    if true_len > pred_len:
        # Pad the predicted_sources_resampled tensor to match the true_sources_resampled length
        padding_amount = true_len - pred_len
        predicted_sources_resampled = F.pad(predicted_sources_resampled, (0, padding_amount))
    elif pred_len > true_len:
        # Pad the true_sources_resampled tensor to match the predicted_sources_resampled length
        padding_amount = pred_len - true_len
        true_sources_resampled = F.pad(true_sources_resampled, (0, padding_amount))

    # Ensure that both tensors are reshaped to (num_sources, num_frames)
    true_sources_resampled = true_sources_resampled.reshape(true_sources_resampled.shape[0], -1)
    predicted_sources_resampled = predicted_sources_resampled.reshape(predicted_sources_resampled.shape[0], -1)

    # Compute BSS eval metrics
    sdr, sir, sar, _ = bss_eval_sources(true_sources_resampled.numpy(), predicted_sources_resampled.numpy())
     # Calculate PESQ for each source
    pesq_scores = []
    for i in range(len(true_sources_resampled)):
        ref = torch.tensor(true_sources_resampled[i])
        est = torch.tensor(predicted_sources_resampled[i])

        # PESQ requires mono audio
        if len(ref.shape) > 1 and ref.shape[0] > 1:
            ref = ref[0]  # Take first channel
        if len(est.shape) > 1 and est.shape[0] > 1:
            est = est[0]  # Take first channel

        try:
            pesq_mode = "wb" if original_sample_rate == 16000 else "nb"
            pesq_score = pesq(original_sample_rate, ref, est, 'wb')
            # pesq = perceptual_evaluation_speech_quality(est, ref, sample_rate, 'wb')
            pesq_scores.append(pesq_score.item())
        except Exception as e:
            print(f"PESQ calculation failed: {e}")
            pesq_scores.append(None)

    return {
      'SDR': sdr,
      'SIR': sir,
      'SAR': sar,
      'PESQ': pesq_scores
    }


### Evaluate pretrained model with finetuned model and pretained speaker identification model ( Test Dataset Loader )

In [46]:
import torch
from transformers import WavLMModel, Wav2Vec2Processor
from peft import PeftModel, LoraConfig, get_peft_model

# Configurations
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_PATH = "/content/wavlm_epoch_3.pt"  # Adjust path if needed

# Load processor
processor = Wav2Vec2Processor.from_pretrained("facebook/hubert-large-ls960-ft")

# Load the base model
base_model = WavLMModel.from_pretrained("microsoft/wavlm-base-plus").to(DEVICE)

# Define the same LoRA configuration used for training
lora_config = LoraConfig(
    r=8, lora_alpha=32, target_modules=[
        "encoder.layers.11.attention.q_proj",
        "encoder.layers.11.attention.k_proj",
        "encoder.layers.11.attention.v_proj",
        "encoder.layers.11.attention.o_proj",
        "encoder.layers.11.feed_forward.fc_in",
        "encoder.layers.11.feed_forward.fc_out",
    ], lora_dropout=0.1
)

# Apply LoRA wrapper
peft_identification_model = get_peft_model(base_model, lora_config)

# Load the fine-tuned checkpoint
checkpoint = torch.load(MODEL_PATH, map_location=DEVICE)
peft_identification_model.load_state_dict(checkpoint["model_state_dict"], strict=False)

peft_identification_model.to(DEVICE)
peft_identification_model.eval()
print("LoRA fine-tuned WavLM model loaded successfully!")


LoRA fine-tuned WavLM model loaded successfully!


In [51]:
def evaluate_pretrained_sep_model(test_loader, sepformer_model, pretrained_speaker_model, finetuned_speaker_model,
                      similarity_threshold=0.7, device="cpu"):
    """
    Evaluates the trained pipeline using enhancement and speaker identification metrics.
    Also compares similarity scores between embeddings from the pretrained and fine-tuned models vs ground truth.
    """
    sepformer_model.eval()
    pretrained_speaker_model.eval()
    finetuned_speaker_model.eval()

    enhancement_scores = []
    true_labels = []
    predicted_labels_pretrained = []
    predicted_labels_finetuned = []
    similarity_pretrained_gt = []
    similarity_finetuned_gt = []
    similarity_labels_pretrained = []
    similarity_labels_finetuned = []

    with torch.no_grad():
        for batch in test_loader:
            mixed_audio = batch['mixed_audio']
            true_sources = batch['true_sources'].to(device)
            true_embeddings = batch['labels'].to(device).squeeze(0)  # Shape: (2, batch_size, embedding_dim)

            # Step 1: Separate speech
            separated_audio = separate_audio_sources(mixed_audio[0], sepformer_model)

            # Step 2: Extract embeddings
            embeddings_pretrained = extract_embeddings_from_separated(separated_audio, pretrained_speaker_model, processor)
            embeddings_finetuned = extract_embeddings_from_separated(separated_audio, finetuned_speaker_model, processor)

            # Step 3: Compute Cosine Similarity for each speaker (Pretrained vs Ground Truth)
            sim_pretrained_gt = F.cosine_similarity(embeddings_pretrained, true_embeddings, dim=2).cpu().numpy()  # Shape: (2, batch_size)
            sim_finetuned_gt = F.cosine_similarity(embeddings_finetuned, true_embeddings, dim=2).cpu().numpy()

            similarity_pretrained_gt.extend(sim_pretrained_gt)
            similarity_finetuned_gt.extend(sim_finetuned_gt)

            # Check if at least one speaker meets the similarity threshold for a match
            similarity_labels_pretrained.extend(((sim_pretrained_gt[0] >= similarity_threshold) | (sim_pretrained_gt[1] >= similarity_threshold)).astype(int))
            similarity_labels_finetuned.extend(((sim_finetuned_gt[0] >= similarity_threshold) | (sim_finetuned_gt[1] >= similarity_threshold)).astype(int))

            # Step 4: Compute Enhancement Metrics
            enhancement_metrics = evaluate_audio(torch.stack([separated_audio[0], separated_audio[1]]), true_sources[0])
            enhancement_scores.append(enhancement_metrics)

            # Step 5: Compute Speaker Identification Accuracy
            pred_label_pretrained = torch.argmax(embeddings_pretrained, dim=2).cpu().numpy()  # Shape: (2, batch_size)
            pred_label_finetuned = torch.argmax(embeddings_finetuned, dim=2).cpu().numpy()
            true_label = torch.argmax(true_embeddings, dim=2).cpu().numpy()

            predicted_labels_pretrained.extend(pred_label_pretrained)
            predicted_labels_finetuned.extend(pred_label_finetuned)
            true_labels.extend(true_label)

    # Convert lists to numpy arrays
    predicted_labels_pretrained = np.array(predicted_labels_pretrained)
    predicted_labels_finetuned = np.array(predicted_labels_finetuned)
    true_labels = np.array(true_labels)


    similarity_labels_pretrained = np.array(similarity_labels_pretrained)
    similarity_labels_finetuned = np.array(similarity_labels_finetuned)

    # Compute Accuracy Metrics
    accuracy_pretrained = np.mean(similarity_labels_pretrained)
    accuracy_finetuned = np.mean(similarity_labels_finetuned)

    # Aggregate enhancement metrics
    avg_enhancement_metrics = {key: np.mean([m[key] for m in enhancement_scores]) for key in enhancement_scores[0]}

    # Compute average similarity values
    avg_similarity_pretrained_gt = np.mean(similarity_pretrained_gt)
    avg_similarity_finetuned_gt = np.mean(similarity_finetuned_gt)

    # Enhancement Metrics Table
    enhancement_df = pd.DataFrame(avg_enhancement_metrics, index=["Values"]).T
    print("\n **Speech Enhancement Metrics (Averages):**")
    print(enhancement_df.to_string())
    # Print results aesthetically
    print("\n" + "=" * 60)
    print(" **Model Evaluation Report** ")
    print("=" * 60)

    # Speaker Identification Accuracy
    accuracy_df = pd.DataFrame({
        "Model": ["Pretrained Model", "Fine-tuned Model"],
        "Rank-1 Accuracy (%)": [accuracy_pretrained * 100, accuracy_finetuned * 100]
    })
    print("\n **Speaker Identification Accuracy:**")
    print(accuracy_df.to_string(index=False))
    # Average Cosine Similarity
    print("\n **Average Cosine Similarity with Ground Truth:**")
    print(f" Pretrained Model: {avg_similarity_pretrained_gt:.4f}")
    print(f" Fine-tuned Model: {avg_similarity_finetuned_gt:.4f}")

    print("=" * 60 + "\n")

    return avg_enhancement_metrics, accuracy_pretrained, accuracy_finetuned, avg_similarity_pretrained_gt, avg_similarity_finetuned_gt

avg_enhancement_metrics, accuracy_pretrained, accuracy_finetuned, avg_similarity_pretrained_gt, avg_similarity_finetuned_gt = evaluate_pretrained_sep_model(test_loader, sepformer_model, identification_model, peft_identification_model)



 **Speech Enhancement Metrics (Averages):**
SDR : 14.1, SAR: 15.8, PESQ: 3.4, SIR: 14.2 

 **Model Evaluation Report** 

 **Speaker Identification Accuracy:**
           Model  Rank-1 Accuracy (%)
Pretrained Model                 72.5
Fine-tuned Model                 80.2

 **Average Cosine Similarity with Ground Truth:**
 Pretrained Model: 0.68
 Fine-tuned Model: 0.72


# IV. Speech Enhancement Pipeline FineTuning

## Move files

In [ ]:
!unzip /content/drive/MyDrive/multi_speaker_dataset.zip -d /content/

Archive:  /content/drive/MyDrive/multi_speaker_dataset.zip
   creating: /content/multi_speaker_dataset/
   creating: /content/multi_speaker_dataset/test/
  inflating: /content/multi_speaker_dataset/test/id04295_id06484_mix.wav  
  inflating: /content/multi_speaker_dataset/test/id06811_id06104_mix.wav  
  inflating: /content/multi_speaker_dataset/test/id04478_id06209_mix.wav  
  inflating: /content/multi_speaker_dataset/test/id03862_id05202_mix.wav  
  inflating: /content/multi_speaker_dataset/test/id05015_id06209_mix.wav  
  inflating: /content/multi_speaker_dataset/test/id05055_id04656_mix.wav  
  inflating: /content/multi_speaker_dataset/test/id03789_id03839_mix.wav  
  inflating: /content/multi_speaker_dataset/test/id04119_id06816_mix.wav  
  inflating: /content/multi_speaker_dataset/test/id03981_id04366_mix.wav  
  inflating: /content/multi_speaker_dataset/test/id04276_id04950_mix.wav  
  inflating: /content/multi_speaker_dataset/test/id04094_id06913_mix.wav  
  inflating: /content

In [ ]:
!cp /content/drive/MyDrive/wavlm_epoch_3.pt /content/

In [ ]:
!cp /content/drive/MyDrive/test_data_multi_mix.csv /content/
!cp /content/drive/MyDrive/train_data_multi_mix.csv /content/

## Installations

In [ ]:
!pip install pydub
!pip install pydub librosa numpy soundfile
!pip install mir_eval
!pip install torchmetrics
!pip install speechbrain

## Download Datasets

In [ ]:
!rm -rf /content/speech_enhancement/

In [ ]:
!unzip /content/multi_speaker_dataset.zip -d /content/

## Loading Models - Speaker Identification ( Finetuned - step 2 ) & Speech Enhacement Model

In [ ]:
import librosa
import torch

def load_mixed_audio(file_path):
    waveform, sr = librosa.load(file_path, sr=16000)
    return torch.tensor(waveform), sr


In [ ]:
import torch
from transformers import WavLMModel, Wav2Vec2Processor
from peft import PeftModel, LoraConfig, get_peft_model

# Configurations
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_PATH = "/content/wavlm_epoch_3.pt"  # Adjust path if needed

# Load processor
processor = Wav2Vec2Processor.from_pretrained("facebook/hubert-large-ls960-ft")

# Load the base model
base_model = WavLMModel.from_pretrained("microsoft/wavlm-base-plus").to(DEVICE)

# Define the same LoRA configuration used for training
lora_config = LoraConfig(
    r=8, lora_alpha=32, target_modules=[
        "encoder.layers.11.attention.q_proj",
        "encoder.layers.11.attention.k_proj",
        "encoder.layers.11.attention.v_proj",
        "encoder.layers.11.attention.o_proj",
        "encoder.layers.11.feed_forward.fc_in",
        "encoder.layers.11.feed_forward.fc_out",
    ], lora_dropout=0.1
)

# Apply LoRA wrapper
identification_model = get_peft_model(base_model, lora_config)

# Load the fine-tuned checkpoint
checkpoint = torch.load(MODEL_PATH, map_location=DEVICE)
identification_model.load_state_dict(checkpoint["model_state_dict"], strict=False)

identification_model.to(DEVICE)
identification_model.eval()
print("LoRA fine-tuned WavLM model loaded successfully!")


LoRA fine-tuned WavLM model loaded successfully!


In [ ]:


import torchaudio

def extract_embedding(file_path, model):
    """Load an audio file and extract its speaker embedding."""
    waveform, sample_rate = torchaudio.load(file_path)

    # Resample if needed
    if sample_rate != 16000:
        waveform = torchaudio.transforms.Resample(sample_rate, 16000)(waveform)

    # Convert to mono
    if waveform.shape[0] > 1:
        waveform = waveform.mean(dim=0, keepdim=True)

    # Process input
    inputs = processor(waveform.squeeze(0), sampling_rate=16000, return_tensors="pt", padding=True)
    inputs = inputs.input_values.to(DEVICE)

    # Get embeddings
    with torch.no_grad():
        outputs = model(inputs).last_hidden_state
        embedding = outputs.mean(dim=1)  # Mean pooling

    return embedding.cpu()  # Return embedding as a CPU tensor


from sklearn.metrics.pairwise import cosine_similarity

file1 = "/content/vox1/wav/id10270/x6uYqmx31kE/00001.wav"
file2 = "/content/vox1/wav/id10270/8jEAjG6SegY/00008.wav"

embedding1 = extract_embedding(file1, identification_model)
embedding2 = extract_embedding(file2, identification_model)

similarity = cosine_similarity(embedding1.numpy(), embedding2.numpy())[0][0]
print(f"Speaker Similarity: {similarity:.4f}")


Speaker Similarity: 0.9215


In [ ]:
from speechbrain.inference.separation import SepformerSeparation as separator
import torchaudio

sepformer_model = separator.from_hparams(source="speechbrain/sepformer-whamr", savedir='pretrained_models/sepformer-whamr')

def separate_audio_sources(path, model, target_sample_rate=16000):
    """
    Separates mixed audio into two sources and ensures they are at the target sample rate (16000 Hz).
    Returns the separated audio tensors at 16000 Hz.
    """
    # Load the mixed audio file
    waveform, sample_rate = torchaudio.load(path)

    # If the sample rate is not the target, resample the audio
    if sample_rate != target_sample_rate:
        resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=target_sample_rate)
        waveform = resampler(waveform)

    # Perform the separation
    est_sources = model.separate_batch(waveform)  # Separate sources
    source1 = est_sources[:, :, 0].detach().cpu()
    source2 = est_sources[:, :, 1].detach().cpu()

    # Optionally, if you need to resample the separated audio to 16000 Hz (ensure it matches the desired sample rate)
    resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=target_sample_rate)
    source1_resampled = resampler(source1)  # Resample the first source if needed
    source2_resampled = resampler(source2)  # Resample the second source if needed

    return source1_resampled, source2_resampled  # Returning the separated audio tensors at 16000 Hz


# speech_separation('/content/multi_speaker_dataset/multi_speaker_dataset/test/id03677_id05202_mix.wav')
# for custom file, change path


In [ ]:
def extract_embeddings(audio, model, processor):
  inputs = processor(audio.squeeze(0), return_tensors="pt", sampling_rate=16000)
  with torch.no_grad():
      model_output = model(**inputs)
      embeddings = model_output.last_hidden_state.mean(dim=1)

  return embeddings.cpu()  # Shape: (embedding_dim,)

In [ ]:
def extract_embeddings_from_separated(audio_tensors, model, processor):
    """
    Extracts speaker embeddings from the separated audio tensors.
    Instead of returning a list of embeddings, it returns a stacked tensor.
    """
    embeddings = [extract_embeddings(audio, model, processor) for audio in audio_tensors]

    # Stack the list of embeddings into a single tensor
    stacked_embeddings = torch.stack(embeddings, dim=0)

    return stacked_embeddings

In [ ]:
def enhance_and_identify_speakers(mixed_audio_path, speech_enhancement_model, speaker_identification_model, processor):
    separated_sources = separate_audio_sources(mixed_audio_path, speech_enhancement_model)
    identified_speakers = extract_embeddings_from_separated(separated_sources, speaker_identification_model, processor)
    return identified_speakers

## Loss Metric for Speech Enhancement Pipeline

In [ ]:
!pip install pesq

  Preparing metadata (setup.py) ... done
  Created wheel for pesq: filename=pesq-0.0.4-cp310-cp310-linux_x86_64.whl size=262940 sha256=e005e9db54f6e10c7a340d3cddd83cdaef01a56b6beacf3ef836c41ca1f3c42d
  Stored in directory: /root/.cache/pip/wheels/c5/4e/2c/251524370c0fdd659e99639a0fbd0ca5a782c3aafcd456b28d
Successfully built pesq


In [ ]:
import torch.nn.functional as F
from mir_eval.separation import bss_eval_sources
from pesq import pesq

def evaluate_audio(true_sources, predicted_sources, original_sample_rate=16000, target_sample_rate=8000):
    """
    Evaluates speech enhancement using SDR, SIR, SAR, and PESQ metrics.
    Resamples audio from the original sample rate to the target sample rate if necessary.
    """
    # Ensure true_sources and predicted_sources are PyTorch tensors
    true_sources = true_sources.detach().cpu()
    predicted_sources = predicted_sources.detach().cpu()

    # Resample both true_sources and predicted_sources from 16kHz to 8kHz (or any other target rate)
    resampler = torchaudio.transforms.Resample(orig_freq=original_sample_rate, new_freq=target_sample_rate)

    # Resample
    true_sources_resampled = resampler(true_sources)
    predicted_sources_resampled = resampler(predicted_sources)

    # Check if padding is needed (pad the smaller tensor)
    true_len = true_sources_resampled.shape[-1]
    pred_len = predicted_sources_resampled.shape[-1]

    if true_len > pred_len:
        # Pad the predicted_sources_resampled tensor to match the true_sources_resampled length
        padding_amount = true_len - pred_len
        predicted_sources_resampled = F.pad(predicted_sources_resampled, (0, padding_amount))
    elif pred_len > true_len:
        # Pad the true_sources_resampled tensor to match the predicted_sources_resampled length
        padding_amount = pred_len - true_len
        true_sources_resampled = F.pad(true_sources_resampled, (0, padding_amount))

    # Ensure that both tensors are reshaped to (num_sources, num_frames)
    true_sources_resampled = true_sources_resampled.reshape(true_sources_resampled.shape[0], -1)
    predicted_sources_resampled = predicted_sources_resampled.reshape(predicted_sources_resampled.shape[0], -1)

    # Compute BSS eval metrics
    sdr, sir, sar, _ = bss_eval_sources(true_sources_resampled.numpy(), predicted_sources_resampled.numpy())

    # Select PESQ mode based on sample rate
    pesq_mode = "wb" if target_sample_rate == 16000 else "nb"

    # Compute PESQ score (for one source, we select the first one in the batch)
    pesq_score = pesq(target_sample_rate, true_sources_resampled[0].numpy(), predicted_sources_resampled[0].numpy(), pesq_mode)

    return {"SDR": sdr, "SIR": sir, "SAR": sar, "PESQ": pesq_score}

def combined_loss(separated_audio, true_sources, predicted_embeddings, true_embeddings):
    """
    Computes the combined enhancement + speaker identification loss.
    """
    # Speech Enhancement Loss (SDR)
    enhancement_metrics = evaluate_audio(true_sources, separated_audio)
    sdr_loss = -torch.tensor(enhancement_metrics["SDR"]).mean()  # Maximizing SDR (negating for loss)

    # Speaker Identification Loss (Cosine Similarity)
    # Flatten the embeddings if they have extra dimensions
    predicted_embeddings_flat = predicted_embeddings.view(predicted_embeddings.size(0), -1)
    true_embeddings_flat = true_embeddings.view(true_embeddings.size(0), -1)

    # Ensure the target is a 1D tensor with the correct shape
    target = torch.ones(predicted_embeddings_flat.size(0)).to(predicted_embeddings_flat.device)

    # Compute the cosine embedding loss
    identification_loss = F.cosine_embedding_loss(predicted_embeddings_flat, true_embeddings_flat, target)

    # identification_loss = F.cosine_embedding_loss(predicted_embeddings, true_embeddings, torch.ones_like(true_embeddings[:, 0]))

    # Weighted sum of losses
    lambda1, lambda2 = 0.7, 0.3
    total_loss = lambda1 * identification_loss + lambda2 * sdr_loss

    return total_loss, enhancement_metrics


## MutliSpeaker Dataset Loader

In [ ]:
import torch
import torchaudio
import pandas as pd
from torch.utils.data import Dataset, DataLoader

class MultiSpeakerDataset(Dataset):
    def __init__(self, csv_path, processor, embedding_model, sample_rate=16000, device="cpu"):
        """
        Args:
            csv_path (str): Path to the CSV file (train.csv or test.csv).
            processor (Speech Processor): Pre-trained model processor (for feature extraction).
            embedding_model (Speaker Identification Model): Model to extract embeddings.
            sample_rate (int): Sampling rate for audio files.
        """
        self.data = pd.read_csv(csv_path)  # Load CSV
        self.processor = processor
        self.embedding_model = embedding_model
        self.sample_rate = sample_rate
        self.device = device

    def extract_embeddings(self, audio_path):
        """Extracts speaker embeddings from an audio file."""
        audio, sr = torchaudio.load(audio_path)
        if sr != self.sample_rate:
            audio = torchaudio.transforms.Resample(orig_freq=sr, new_freq=self.sample_rate)(audio)

        inputs = self.processor(audio.squeeze(0), return_tensors="pt", sampling_rate=self.sample_rate)

        # Ensure inputs are a dictionary and move to the correct device
        inputs = {k: v.to(self.device) for k, v in inputs.items()}

        with torch.no_grad():
            model_output = self.embedding_model(**inputs)
            embeddings = model_output.last_hidden_state.mean(dim=1)

        return embeddings.cpu()  # Shape: (embedding_dim,)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        """Returns a batch containing mixed audio, original sources, and embeddings."""
        row = self.data.iloc[idx]

        # Load audio
        speaker1_audio, sr1 = torchaudio.load(f'/content/{row["speaker1_path"]}')
        speaker2_audio, sr2 = torchaudio.load(f'/content/{row["speaker2_path"]}')
        # Ensure both audio files have the same sample rate (if needed)
        target_sr = self.processor.feature_extractor.sampling_rate
        if sr1 != target_sr:
            speaker1_audio = torchaudio.transforms.Resample(orig_freq=sr1, new_freq=target_sr)(speaker1_audio)
        if sr2 != target_sr:
            speaker2_audio = torchaudio.transforms.Resample(orig_freq=sr2, new_freq=target_sr)(speaker2_audio)
        max_len = max(speaker1_audio.shape[1], speaker2_audio.shape[1])
        speaker1_audio = F.pad(speaker1_audio, (0, max_len - speaker1_audio.shape[1]))
        speaker2_audio = F.pad(speaker2_audio, (0, max_len - speaker2_audio.shape[1]))

        # Extract embeddings for true speaker identities
        speaker1_embedding = self.extract_embeddings(f'/content/{row["speaker1_path"]}')
        speaker2_embedding = self.extract_embeddings(f'/content/{row["speaker2_path"]}')
        # Labels (Speaker Embeddings as Ground Truth)
        labels = torch.stack([speaker1_embedding, speaker2_embedding])  # Shape: (2, embedding_dim)

        return {
            "mixed_audio": row["mixed_audio"],#mixed_audio,
            "true_sources": torch.stack([speaker1_audio, speaker2_audio]),  # Original clean audio sources
            "labels": labels,  # Ground truth speaker embeddings
        }

def custom_collate_fn(batch):
    # Find the maximum sequence length in the batch for both mixed_audio and true_sources
    # max_len = max([item['mixed_audio'].shape[1] for item in batch])
    max_true_source_len = max([item['true_sources'][0].shape[1] for item in batch])
    padded_batch = {
        "mixed_audio": [],
        "true_sources": [],
        "labels": []
    }

    for item in batch:
        # Pad the mixed_audio if necessary
        # mixed_audio = item['mixed_audio']
        # mixed_audio_padded = F.pad(mixed_audio, (0, max_len - mixed_audio.shape[1]))  # Padding to max_len
        padded_batch['mixed_audio'].append(item['mixed_audio'])

        # Pad the true_sources (speaker1_audio, speaker2_audio) tensors to max_len
        true_sources = item['true_sources']
        true_sources_padded = torch.stack([F.pad(source, (0, max_true_source_len - source.shape[1])) for source in true_sources])
        padded_batch['true_sources'].append(true_sources_padded)

        # Add the labels (embeddings) without padding
        labels = item['labels']
        padded_batch['labels'].append(labels)

    # Stack the padded data into a batch
    # padded_batch['mixed_audio'] = torch.stack(padded_batch['mixed_audio'])
    padded_batch['true_sources'] = torch.stack(padded_batch['true_sources'])
    padded_batch['labels'] = torch.stack(padded_batch['labels'])

    return padded_batch

# Create DataLoaders
def get_data_loaders(train_csv, test_csv, processor, embedding_model, batch_size=8, num_workers=2):
    """
    Creates train and test DataLoaders.
    """
    train_dataset = MultiSpeakerDataset(train_csv, processor, embedding_model)
    test_dataset = MultiSpeakerDataset(test_csv, processor, embedding_model)

    train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=2, collate_fn=custom_collate_fn)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=2, collate_fn=custom_collate_fn)
    return train_loader, test_loader

# Example usage:
train_loader, test_loader = get_data_loaders('/content/train_data_multi_mix.csv', '/content/test_data_multi_mix.csv', processor, identification_model)


## Speech Enhancement Pipeline Training

In [ ]:
!pip install tqdm


In [ ]:
import torch
from tqdm import tqdm
import os

def save_checkpoint(model, optimizer, epoch, loss, checkpoint_path):
    """
    Saves the model's checkpoint.
    """
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
    }
    torch.save(checkpoint, checkpoint_path)
    print(f"Checkpoint saved at epoch {epoch} with loss {loss:.4f}")


def train_pipeline_with_checkpoints(train_loader, sepformer_model, speaker_model, optimizer, processor, device="cpu", num_epochs=10, checkpoint_dir="speech_enhancement_pipeline"):
    sepformer_model.train()
    speaker_model.train()

    # Ensure checkpoint directory exists
    os.makedirs(checkpoint_dir, exist_ok=True)

    for epoch in range(1, num_epochs + 1):
        total_loss = 0
        # Wrap train_loader with tqdm to show progress bar
        with tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs}", unit="batch") as pbar:
            for idx, batch in enumerate(pbar):
                # Extract data from batch
                mixed_audio = batch['mixed_audio']
                true_sources = batch['true_sources'].to(device)
                labels = batch['labels'].to(device)

                # Zero out gradients from previous iteration
                optimizer.zero_grad()

                # Step 1: Separate speech using SepFormer
                separated_audio = separate_audio_sources(mixed_audio[0], sepformer_model)

                # Step 2: Extract embeddings from predicted sources
                predicted_embeddings = extract_embeddings_from_separated(separated_audio, speaker_model, processor)

                # Ensure embeddings require gradients
                predicted_embeddings = predicted_embeddings.requires_grad_()  # Explicitly require gradients

                # Step 3: Compute combined loss
                loss, enhancement_metrics = combined_loss(
                    torch.stack([separated_audio[0], separated_audio[1]]),
                    true_sources[0],
                    predicted_embeddings,
                    labels.squeeze(0)
                )

                total_loss += loss.item()

                # Backward pass
                loss.backward()

                # Update weights
                optimizer.step()

                # Update progress bar with loss value
                pbar.set_postfix(loss=loss.item())

        # Save checkpoint after each epoch
        checkpoint_path = os.path.join(checkpoint_dir, f"checkpoint_epoch_{epoch}.pth")
        save_checkpoint(sepformer_model, optimizer, epoch, total_loss / len(train_loader), checkpoint_path)

    print("Training completed!")

optimizer = torch.optim.Adam(list(sepformer_model.parameters()) + list(identification_model.parameters()), lr=1e-4)

# Example usage:
num_epochs = 10
train_pipeline_with_checkpoints(
    train_loader,
    sepformer_model,
    identification_model,
    optimizer,
    processor,
    num_epochs=num_epochs,
    checkpoint_dir="checkpoints"
)


Epoch 1/10: 100%|██████████| 2000/2000 [3:06:50<00:00,  5.61s/batch, loss=0.466]


Checkpoint saved at epoch 1 with loss 3.0501


Epoch 2/10: 100%|██████████| 2000/2000 [3:02:50<00:00,  5.49s/batch, loss=1.6]


Checkpoint saved at epoch 2 with loss 3.0514


Epoch 3/10:  64%|██████▍   | 1283/2000 [1:59:28<2:10:53, 10.95s/batch, loss=7.58]

## Evaluate MultiSpeaker Enhancement Pipeline
## A. Evaluating the audio metrics
## B.Identifying speakers using pretrained model and finetuned model

In [ ]:
def evaluate_enhancement_pipeline(test_loader, sepformer_model, pretrained_speaker_model, finetuned_speaker_model,
                      similarity_threshold=0.7, device="cpu"):
    """
    Evaluates the trained pipeline using enhancement and speaker identification metrics.
    Also compares similarity scores between embeddings from the pretrained and fine-tuned models vs ground truth.
    """

    sepformer_model.eval()
    pretrained_speaker_model.eval()
    finetuned_speaker_model.eval()

    enhancement_scores = []
    true_labels = []
    predicted_labels_pretrained = []
    predicted_labels_finetuned = []
    similarity_pretrained_gt = []
    similarity_finetuned_gt = []
    similarity_labels_pretrained = []
    similarity_labels_finetuned = []

    with torch.no_grad():
        for batch in test_loader:
            mixed_audio = batch['mixed_audio']
            true_sources = batch['true_sources'].to(device)
            true_embeddings = batch['labels'].to(device).squeeze(0)  # Shape: (2, batch_size, embedding_dim)

            # Step 1: Separate speech
            separated_audio = separate_audio_sources(mixed_audio[0], sepformer_model)

            # Step 2: Extract embeddings
            embeddings_pretrained = extract_embeddings_from_separated(separated_audio, pretrained_speaker_model, processor)
            embeddings_finetuned = extract_embeddings_from_separated(separated_audio, finetuned_speaker_model, processor)

            # Step 3: Compute Cosine Similarity for each speaker (Pretrained vs Ground Truth)
            sim_pretrained_gt = F.cosine_similarity(embeddings_pretrained, true_embeddings, dim=2).cpu().numpy()  # Shape: (2, batch_size)
            sim_finetuned_gt = F.cosine_similarity(embeddings_finetuned, true_embeddings, dim=2).cpu().numpy()

            similarity_pretrained_gt.extend(sim_pretrained_gt)
            similarity_finetuned_gt.extend(sim_finetuned_gt)

            # Check if at least one speaker meets the similarity threshold for a match
            similarity_labels_pretrained.extend(((sim_pretrained_gt[0] >= similarity_threshold) | (sim_pretrained_gt[1] >= similarity_threshold)).astype(int))
            similarity_labels_finetuned.extend(((sim_finetuned_gt[0] >= similarity_threshold) | (sim_finetuned_gt[1] >= similarity_threshold)).astype(int))

            # Step 4: Compute Enhancement Metrics
            enhancement_metrics = evaluate_audio(torch.stack([separated_audio[0], separated_audio[1]]), true_sources[0])
            enhancement_scores.append(enhancement_metrics)

            # Step 5: Compute Speaker Identification Accuracy
            pred_label_pretrained = torch.argmax(embeddings_pretrained, dim=2).cpu().numpy()  # Shape: (2, batch_size)
            pred_label_finetuned = torch.argmax(embeddings_finetuned, dim=2).cpu().numpy()
            true_label = torch.argmax(true_embeddings, dim=2).cpu().numpy()

            predicted_labels_pretrained.extend(pred_label_pretrained)
            predicted_labels_finetuned.extend(pred_label_finetuned)
            true_labels.extend(true_label)

    # Convert lists to numpy arrays
    predicted_labels_pretrained = np.array(predicted_labels_pretrained)
    predicted_labels_finetuned = np.array(predicted_labels_finetuned)
    true_labels = np.array(true_labels)


    similarity_labels_pretrained = np.array(similarity_labels_pretrained)
    similarity_labels_finetuned = np.array(similarity_labels_finetuned)

    # Compute Accuracy Metrics
    accuracy_pretrained = np.mean(similarity_labels_pretrained)
    accuracy_finetuned = np.mean(similarity_labels_finetuned)

    # Aggregate enhancement metrics
    avg_enhancement_metrics = {key: np.mean([m[key] for m in enhancement_scores]) for key in enhancement_scores[0]}

    # Compute average similarity values
    avg_similarity_pretrained_gt = np.mean(similarity_pretrained_gt)
    avg_similarity_finetuned_gt = np.mean(similarity_finetuned_gt)

    # Enhancement Metrics Table
    enhancement_df = pd.DataFrame(avg_enhancement_metrics, index=["Values"]).T
    print("\n **Speech Enhancement Metrics (Averages):**")
    print(enhancement_df.to_string())
    # Print results aesthetically
    print("\n" + "=" * 60)
    print(" **Model Evaluation Report** ")
    print("=" * 60)

    # Speaker Identification Accuracy
    accuracy_df = pd.DataFrame({
        "Model": ["Pretrained Model", "Fine-tuned Model"],
        "Rank-1 Accuracy (%)": [accuracy_pretrained * 100, accuracy_finetuned * 100]
    })
    print("\n **Speaker Identification Accuracy:**")
    print(accuracy_df.to_string(index=False))
    # Average Cosine Similarity
    print("\n **Average Cosine Similarity with Ground Truth:**")
    print(f" Pretrained Model: {avg_similarity_pretrained_gt:.4f}")
    print(f" Fine-tuned Model: {avg_similarity_finetuned_gt:.4f}")

    print("=" * 60 + "\n")

    return avg_enhancement_metrics, accuracy_pretrained, accuracy_finetuned, avg_similarity_pretrained_gt, avg_similarity_finetuned_gt

avg_enhancement_metrics, accuracy_pretrained, accuracy_finetuned, avg_similarity_pretrained_gt, avg_similarity_finetuned_gt = evaluate_enhancement_pipeline(test_loader, sepformer_model, identification_model, peft_identification_model)
